In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from collections import Counter
import re, hashlib

PATH = Path("../data/processed/corpus_politico_codificado_utf8.csv")

# Leer el archivo
df = pd.read_csv(PATH, encoding="utf-8", engine="python")
df.columns = [c.strip().lower() for c in df.columns]

# ✅ Ajuste para tu dataset
if set(df.columns) == {"texto","etiqueta"}:
    df.rename(columns={"texto":"text","etiqueta":"labels"}, inplace=True)

elif set(df.columns) >= {"text","labels_name","labels"}:
    df = df[["text","labels"]]

elif set(df.columns) == {"text","labels"}:
    pass

else:
    raise AssertionError(f"Encabezado inesperado: {list(df.columns)}. Debe tener ['texto','etiqueta'] o ['text','labels'].")

# Convertir tipo de datos
df["text"] = df["text"].astype(str).str.strip()
df["labels"] = df["labels"].astype(str).str.strip()

# Mapear etiquetas de texto a número si hace falta
if pd.api.types.is_string_dtype(df["labels"]):
    # si son textos como 'logos' o 'ad_hominem'
    if df["labels"].str.isalpha().any():
        etiquetas = sorted(df["labels"].unique())
        mapa = {etq:i for i,etq in enumerate(etiquetas)}
        df["labels"] = df["labels"].map(mapa)
        print("📘 Mapeo de etiquetas:", mapa)
    else:
        # si ya son '0', '1', '2', '3' pero en string
        df["labels"] = df["labels"].astype(int)


df["labels"] = df["labels"].astype(int)

print("✅ Estructura OK | Filas:", len(df))
print("Distribución:", Counter(df["labels"]))
display(df.head(5))


✅ Estructura OK | Filas: 1000
Distribución: Counter({0: 250, 3: 250, 2: 250, 1: 250})


,text,labels
0,"Ese candidato solo sabe hablar, pero nunca ha ...",0
1,Quienes nos critican desde Lima no entienden l...,0
2,"Habla de moral, pero no puede explicar de qué ...",0
3,Ese congresista cambió de partido más veces qu...,0
4,"Siempre promete cambios, pero ni siquiera camb...",0


In [2]:
print(df["labels"].unique())


[0 3 2 1]


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np, re

# --- 1. Normalización básica (acentos, espacios, símbolos) ---
def normalize_text(s: str) -> str:
    s = str(s).lower().strip()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[\"“”‘’]+", "", s)
    s = re.sub(r"[^\w\sáéíóúüñ]", "", s)
    return s

df["norm_text"] = df["text"].apply(normalize_text)

# --- 2. Duplicados exactos ---
before = len(df)
df = df.drop_duplicates(subset=["norm_text","labels"]).reset_index(drop=True)
print(f"🧹 Duplicados exactos eliminados: {before - len(df)} | Quedan: {len(df)}")

# --- 3. Casi-duplicados (similitud > 0.9) ---
tfidf = TfidfVectorizer(min_df=1, max_features=5000).fit_transform(df["norm_text"])
sim = cosine_similarity(tfidf)

mask = np.triu(np.ones(sim.shape), k=1).astype(bool)
pairs = np.argwhere(sim > 0.9)
keep = set()
for i,j in pairs:
    if i not in keep and j not in keep:
        keep.add(j)  # elimina la segunda

print(f"🔍 Casi-duplicados detectados: {len(keep)}")

df_clean = df.drop(list(keep)).reset_index(drop=True)
print(f"✅ Dataset final sin duplicados: {len(df_clean)} filas")

# --- 4. Guardado ---
out = Path("../data/processed/clean_v2/corpus_politico_codificado_utf8_clean_v2.csv")
out.parent.mkdir(parents=True, exist_ok=True)
df_clean[["text","labels"]].to_csv(out, index=False, encoding="utf-8")
print("💾 Guardado en:", out)


🧹 Duplicados exactos eliminados: 6 | Quedan: 994
🔍 Casi-duplicados detectados: 994
✅ Dataset final sin duplicados: 0 filas
💾 Guardado en: ..\data\processed\clean_v2\corpus_politico_codificado_utf8_clean_v2.csv


In [4]:
# --- 3. Casi-duplicados (similitud > 0.95) ---
tfidf = TfidfVectorizer(min_df=1, max_features=5000).fit_transform(df["norm_text"])
sim = cosine_similarity(tfidf)

pairs = np.argwhere(sim > 0.95)  # sube el umbral
eliminar = set()
for i, j in pairs:
    if i != j and j not in eliminar:
        eliminar.add(j)

print(f"🔍 Casi-duplicados detectados: {len(eliminar)}")

df_clean = df.drop(list(eliminar)).reset_index(drop=True)
print(f"✅ Dataset final sin duplicados: {len(df_clean)} filas")

# --- 4. Guardado ---
out = Path("../data/processed/clean_v2/corpus_politico_codificado_utf8_clean_v2.csv")
out.parent.mkdir(parents=True, exist_ok=True)
df_clean[["text","labels"]].to_csv(out, index=False, encoding="utf-8")
print("💾 Guardado en:", out)


🔍 Casi-duplicados detectados: 4
✅ Dataset final sin duplicados: 990 filas
💾 Guardado en: ..\data\processed\clean_v2\corpus_politico_codificado_utf8_clean_v2.csv


In [5]:
import pandas as pd
from pathlib import Path

# Carga el dataset limpio
df = pd.read_csv("../data/processed/clean_v2/corpus_politico_codificado_utf8_clean_v2.csv")

# Mezcla aleatoria reproducible
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Verifica orden
print(df.head(10))
print("Distribución:", df["labels"].value_counts().to_dict())

# Guarda barajado
out_path = Path("../data/processed/clean_v2/corpus_politico_codificado_utf8_clean_v2_shuffled.csv")
df.to_csv(out_path, index=False, encoding="utf-8")
print(f"✅ Dataset barajado guardado en: {out_path}")


                                                text  labels
0             Habla de verdad y solo cuenta su parte       0
1  La fuerza del pueblo es más fuerte que cualqui...       3
2  Más del 60 % de los empleos creados en pandemi...       2
3          No hay fuerza más grande que la esperanza       3
4  Se llena la boca de patria y vacía los bolsill...       0
5  Más de 50 000 escolares abandonaron clases tra...       2
6      El sol volverá a brillar sobre nuestra tierra       3
7  Dice que ama al país, pero lo ve como un negoc...       0
8  Se dice ejemplo, pero su historia está llena d...       0
9  El 45 % de los jóvenes peruanos trabaja sin co...       2
Distribución: {0: 250, 2: 250, 3: 246, 1: 244}
✅ Dataset barajado guardado en: ..\data\processed\clean_v2\corpus_politico_codificado_utf8_clean_v2_shuffled.csv


In [6]:
from pathlib import Path
import pandas as pd
import numpy as np
import re, hashlib
from collections import Counter

# 1) cargar el barajado
DATA = Path("../data/processed/clean_v2/corpus_politico_codificado_utf8_clean_v2_shuffled.csv")
df = pd.read_csv(DATA, encoding="utf-8")
assert {"text","labels"}.issubset(df.columns), df.columns

SEED = 42
N_FOLDS = 5

# 2) group_id para evitar fuga (coloca en mismo fold textos casi-iguales)
def norm_for_group(s: str) -> str:
    s = str(s).lower()
    s = re.sub(r"\d+%","<pct>", s)
    s = re.sub(r"\d+","<num>", s)
    s = re.sub(r"[^\w\sáéíóúüñ]"," ", s)
    s = re.sub(r"\s+"," ", s).strip()
    return s

df["group_id"] = df["text"].apply(lambda t: hashlib.md5(norm_for_group(t).encode()).hexdigest())

# 3) StratifiedGroupKFold si está disponible; si no, fallback a StratifiedKFold
try:
    from sklearn.model_selection import StratifiedGroupKFold
    sgkf = StratifiedGroupKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
    splitter = sgkf.split(df, y=df["labels"], groups=df["group_id"])
except Exception:
    from sklearn.model_selection import StratifiedKFold
    print("⚠️ scikit-learn sin StratifiedGroupKFold; usando StratifiedKFold (sin grupos).")
    sgkf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
    splitter = sgkf.split(df, y=df["labels"])

# 4) asignar fold a cada fila
df["fold"] = -1
for fold_id, (train_idx, val_idx) in enumerate(splitter):
    df.loc[val_idx, "fold"] = fold_id
assert (df["fold"]>=0).all(), "Hay filas sin fold."

# 5) reporte rápido
print("Distribución global:", Counter(df["labels"]))
print("Por fold (val):")
for k in range(N_FOLDS):
    print(k, Counter(df[df["fold"]==k]["labels"]))

# 6) exportar: (a) un solo CSV con fold, (b) pares train/val por fold
OUT_DIR = Path("../data/processed/clean_v2/folds")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# (a) maestro con fold
master_path = OUT_DIR / "corpus_clean_v2_folds.csv"
df.to_csv(master_path, index=False, encoding="utf-8")
print("✅ Guardado maestro con folds:", master_path)

# (b) archivos por fold
for k in range(N_FOLDS):
    train_df = df[df["fold"]!=k][["text","labels"]].reset_index(drop=True)
    val_df   = df[df["fold"]==k][["text","labels"]].reset_index(drop=True)
    train_path = OUT_DIR / f"train_fold{k}.csv"
    val_path   = OUT_DIR / f"val_fold{k}.csv"
    train_df.to_csv(train_path, index=False, encoding="utf-8")
    val_df.to_csv(val_path, index=False, encoding="utf-8")
    print(f"💾 fold {k}: {len(train_df)} train | {len(val_df)} val  ->  {train_path.name} / {val_path.name}")


Distribución global: Counter({0: 250, 2: 250, 3: 246, 1: 244})
Por fold (val):
0 Counter({0: 60, 3: 52, 1: 46, 2: 40})
1 Counter({3: 55, 0: 51, 2: 51, 1: 41})
2 Counter({2: 52, 0: 52, 1: 51, 3: 43})
3 Counter({1: 57, 2: 50, 3: 46, 0: 45})
4 Counter({2: 57, 3: 50, 1: 49, 0: 42})
✅ Guardado maestro con folds: ..\data\processed\clean_v2\folds\corpus_clean_v2_folds.csv
💾 fold 0: 792 train | 198 val  ->  train_fold0.csv / val_fold0.csv
💾 fold 1: 792 train | 198 val  ->  train_fold1.csv / val_fold1.csv
💾 fold 2: 792 train | 198 val  ->  train_fold2.csv / val_fold2.csv
💾 fold 3: 792 train | 198 val  ->  train_fold3.csv / val_fold3.csv
💾 fold 4: 792 train | 198 val  ->  train_fold4.csv / val_fold4.csv
